In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio import AlignIO
import sys
import pymol

In [2]:
#fonction renvoyant un dot_plot (matrice de point) á partir de deux sequences avec une fenêtre comme seuil minimum
def dotplot(seq1,seq2,window):
    x=[]
    y=[]
    for i in range(len(seq1)-window):
        for j in range(len(seq2)-window):
            if seq1[i:i+window]==seq2[j:j+window]:
                x.append(i)
                y.append(j)
    return plt.scatter(x,y)

In [3]:
#fonction renvoyant les coordonnées des points du dot_plot (matrice de point) á partir de deux sequences avec une fenêtre comme seuil minimum
def dotplot_index(seq1,seq2,window):
    x=[]
    y=[]
    for i in range(len(seq1)-window):
        for j in range(len(seq2)-window):
            if seq1[i:i+window]==seq2[j:j+window]:
                x.append(i)
                y.append(j)
    return x,y

In [4]:
# fonction faisant un alignement multiple à partir du fichier file par le programme Clustalw2.exe, produisant en sortie un fichier file.aln et file.dnd
def msalign(file):
    clustalw2=r"C:\Program Files (x86)\ClustalW2\clustalw2.exe"
    clustalw_cline = ClustalwCommandline(clustalw2, infile= file)
    stdout, stderr = clustalw_cline()    


In [5]:
# Fonction pour afficher les alignements
def display_alignment(file):
    clustalalign = AlignIO.read(f"{file[:-6]}.aln", "clustal")    
    dico_aa={'G':0,'P':1,'A':2,'V':3,'L':4,'I':5,'M':6,'C':7,'F':8,'Y':9,'W':10,'H':11,'K':12,'R':13,'Q':14,'N':15,'E':16,'D':17,'S':18,'T':19,'-':20}
    for a in clustalalign:
        for i in a.seq:
            if i not in dico_aa.keys():
                print(i)
            
        print("*"*50)
        print(a.description)
        print(a.seq)
        print(len(a.seq))

In [9]:
# fonction créant un graphique représentant la conservation à chaque position de la séquence consensus et retournant le dataframe AA/score et la sequence
def graphique_conservation(file,seuil=0):
        clustalalign = AlignIO.read(f"{file[:-6]}.aln", "clustal")    
        length=len(clustalalign[0].seq)
        dico_aa={'G':0,'P':1,'A':2,'V':3,'L':4,'I':5,'M':6,'C':7,'F':8,'Y':9,'W':10,'H':11,'K':12,'R':13,'Q':14,'N':15,'E':16,'D':17,'S':18,'T':19,'-':20}
        array_aa=['G','P','A','V','L','I','M','C','F','Y','W','H','K','R','Q','N','E','D','S','T','-']
        alignment_matrix=np.zeros((21,length))
        for s in range(len(clustalalign)):
            for a in range(len(clustalalign[s].seq)):
                if clustalalign[s].seq[a] in dico_aa.keys():
                    id_row=dico_aa[clustalalign[s].seq[a]]
                    alignment_matrix[id_row][a]+=1 

        best_aa=[]
        for n in alignment_matrix.argmax(axis=0):
            best_aa.append(array_aa[n])

        score_aa=[]
        for b,j in zip(alignment_matrix.argmax(axis=0),range(length)):
            score_aa.append(alignment_matrix[b][j]/len(clustalalign))
        
        data=pd.DataFrame({'AA':best_aa,'score':score_aa})

        list_color=[]
        for a in data['AA']:
            list_color.append(dico_aa[a])
        dico_color={'G':'#FADBD8','P':'#A9CCE3','A':'#DAEA0C','V':'#F9E79F','L':'#F0B27A','I':'#85929E','M':'#EC7063','C':'#8E44AD','F':'#5DADE2','Y':'#EA0CCF','W':'#F1C40F','H':'#E67E22','K':'#E74C3C','R':'#1E8449','Q':'#B7950B','N':'#1A5276','E':'#1E8449','D':'#0CD8EA','S':'#4A235A','T':'#6E2C00','-':20}
    
        plt.figure(figsize=(25,10))
        plt.ylabel("conservation en %")
        plt.xlabel('position')
        plt.legend(loc=3, prop={'size': 8})
        list_label=[]
        for i in range(length):
            color=dico_color[data.loc[i]['AA']]
            if data.loc[i]['score']>seuil:
                if data.iloc[i]['AA'] not in list_label:
                    plt.scatter(i,data.loc[i]['score'],s=data.loc[i]['score']*1000,cmap='magma',label=data.loc[i]['AA'],c=color)
                    list_label.append(data.loc[i]['AA'])
                else:
                    plt.scatter(i,data.loc[i]['score'],s=data.loc[i]['score']*1000,cmap='magma',c=color)

        params = {'legend.fontsize': 20,
                  'legend.handlelength': 2}
        plot.rcParams.update(params) 
        plt.legend()
        return data,best_aa

In [11]:
# Fonction qui calcule la RMSD locale à chaque position pour une fenêtre déterminée en paramètres pour pdb1 superposée sur pdb2, et qui retourne la liste des RMSD et trace un graphique
# windoxw: taille de la fenêtre; pdb1/pdb2: nom fichier pdb; length_seq1: taille de la séquence de la protéine à analyser
def local_rmsd(window,pdb1,pdb2,length_seq1):
    
    # Open a PyMOL window
    _stdouterr = sys.stdout, sys.stderr
    pymol.finish_launching(['pymol', '-q'])
    sys.stdout, sys.stderr = _stdouterr

    list_RMSD = []
    pymol.cmd.load(pdb2)

    for i in range(length_seq1):
        pymol.cmd.load(pdb1)
        s=pymol.cmd.select(f"{pdb1[:-4]} and resid {i}-{i+window}")
        pymol.cmd.extract("obj1","s")
        result=pymol.cmd.cealign("obj1",f"{pdb2[:-4]}")
        list_RMSD.append(result["RMSD"])
        pymol.cmd.delete("obj1")
        pymol.cmd.delete(f"{pdb1[:-4]}")
        
    plt.plot(list_RMSD)
    return list_RMSD


    

In [ ]:
#Fonction créant un dictionnaire avec pour clé l'id et pour valeur les GO term des voies biologiques
def dico_bp(list_id):
    for i in list_id:
        dico_bp_b={}
        list_temp=[]
        r=requests.get(f"http://www.ebi.ac.uk/QuickGO/services/annotation/search?geneProductId={i}")
        data=r.json()
        for j in data["results"]:
            if j["goAspect"]=="biological_process":
                list_temp.append(j["goId"])
        dico_bp_b[i]=list_temp
    

In [ ]:
def common_parent_go_ids(terms, go):
    '''
        This function finds the common ancestors in the GO 
        tree of the list of terms in the input.
    '''
    # Find candidates from first
    rec = go[terms[0]]
    candidates = rec.get_all_parents()
    candidates.update({terms[0]})
    
    # Find intersection with second to nth term
    for term in terms[1:]:
        rec = go[term]
        parents = rec.get_all_parents()
        parents.update({term})
        
        # Find the intersection with the candidates, and update.
        candidates.intersection_update(parents)
        
    return candidates

In [ ]:
def deepest_common_ancestor(terms, go):
    '''
        This function gets the nearest common ancestor 
        using the above function.
        Only returns single most specific - assumes unique exists.
    '''
    # Take the element at maximum depth. 
    return max(common_parent_go_ids(terms, go), key=lambda t: go[t].depth)

In [ ]:
def convert_ACC_ENSEMBL(id):
    url = 'http://www.uniprot.org/mapping/' 
    params = {'from':'ACC','to':'ENSEMBL_ID','format':'tab','query':id} 
    request = requests.get(url, params)
    return request.text.split("\n")[1].split("\t")[1]


In [ ]:
def convert_ENSEMBL_ACC(id):
    url = 'http://www.uniprot.org/mapping/' 
    params = {'from':'ENSEMBL_ID','to':'ACC','format':'tab','query':id} 
    request = requests.get(url, params)
    return request.text.split("\n")[1].split("\t")[1]


In [ ]:
def convert_ACC_GENENAME(id):
    url = 'http://www.uniprot.org/mapping/' 
    params = {'from':'ACC','to':'GENENAME','format':'tab','query':id} 
    request = requests.get(url, params)
    return request.text.split("\n")[1].split("\t")[1]
